# Work with Entities

**[Entities](https://docs.aperturedata.io/category/entity-commands) are a way to represent any [application level concept](https://docs.aperturedata.io/concepts/Schema) in ApertureDB like a Person, Ingredient, Product, Event, etc**

## Connect to the database

If you haven't already setup the database or configured it, check out our [quick start guide](https://docs.aperturedata.io/Setup/Quick%20Start)

In [1]:
from aperturedb.CommonLibrary import create_connector

# Create the connector for ApertureDB
client = create_connector()

In [2]:
# Simple query to see how the database is doing
# https://docs.aperturedata.io/query_language/Reference/db_commands/GetStatus
query = [{
    "GetStatus": {
    }
}]

# Execute the query to get back a JSON response for GetStatus 
response, blobs = client.query(query)

client.print_last_response()

[
    {
        "GetStatus": {
            "info": "OK",
            "status": 0,
            "system": "ApertureDB",
            "version": "0.17.23"
        }
    }
]


## Create or Add an Ingredient to ApertureDB

Let's say we want to introduce some ingredients that we use to cook. One way to introduce new entities in the database is through our query language

**For bulk additions, we recommend using the [Python SDK loaders](http://docs.aperturedata.io/category/manage-multimodal-datasets)**

In [3]:
query = [{
    "AddEntity": {
        "class": "Ingredient",           # Every entity belongs to a class, in this case Ingredient
        "properties": {                  # You can define as many key, value properties per entity, as needed
            "name": "butter",            # The data type for "name" property of an Ingredient will now always be of the type "string".
            "macronutrient": "fat",
            "subgroup": "dairy",
            "category": "vegetarian"
        },
        "if_not_found": {                # conditional add
            "name": ["==", "butter"] 
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "AddEntity": {
            "status": 0
        }
    }
]


## Read or Query properties of the added Ingredient

Verify this "Ingredient" entity was added to the database and read all the property values

In [4]:
query = [{
    "FindEntity": {
        "with_class": "Ingredient",               # Every entity belongs to a class, in this case Ingredient
        "constraints": {
            "name": ["==", "butter"],
            "subgroup": ["==", "dairy"] 
        },
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "FindEntity": {
            "entities": [
                {
                    "_uniqueid": "23.0.224300",
                    "category": "vegetarian",
                    "macronutrient": "fat",
                    "name": "butter",
                    "subgroup": "dairy"
                }
            ],
            "returned": 1,
            "status": 0
        }
    }
]


## Update properties of the Ingredient already in ApertureDB

Use [UpdateEntity](https://docs.aperturedata.io/query_language/Reference/entity_commands/UpdateEntity) if any of the attributes need a new value or your application now needs a new attribute in existing entities

In [5]:
query = [{
    "UpdateEntity": {
        "with_class": "Ingredient",
        "properties": {
            "micronutrient": "omega-3"     # property will get added if missing or the value will be updated
        },
        "constraints": {
            "name": ["==", "butter"]
        },
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "UpdateEntity": {
            "count": 1,
            "status": 0
        }
    }
]


In [6]:
query = [{
    "FindEntity": {
        "with_class": "Ingredient",
        "constraints": {
            "name": ["==", "butter"]                # property names and values are case sensitive
        },
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "FindEntity": {
            "entities": [
                {
                    "_uniqueid": "23.0.224300",
                    "category": "vegetarian",
                    "macronutrient": "fat",
                    "micronutrient": "omega-3",
                    "name": "butter",
                    "subgroup": "dairy"
                }
            ],
            "returned": 1,
            "status": 0
        }
    }
]


## Delete the Ingredient we no longer need

In [7]:
query = [{
    "DeleteEntity": {
        "with_class": "Ingredient",
        "constraints": {
            "name": ["==", "butter"] 
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "DeleteEntity": {
            "count": 1,
            "status": 0
        }
    }
]


### Verify deletion

We can verify that the entity is not longer in the database with another FindEntity

In [8]:
query = [{
    "FindEntity": {
        "with_class": "Ingredient",
        "constraints": {
            "name": ["==", "butter"] 
        },
        "results": {
            "all_properties": True
        }
    }
}]

response, blobs = client.query(query)

client.print_last_response()

[
    {
        "FindEntity": {
            "returned": 0,
            "status": 0
        }
    }
]
